In [1]:
import torch
import matplotlib.pyplot as plt
import struct
from collections import  defaultdict

from transformers import GPT2LMHeadModel
from transformers import AutoTokenizer
from transformers import LlamaForCausalLM
from transformers import T5ForConditionalGeneration

import tqdm

In [2]:
def get_exponent(model, nbits):
    exponent = {}
    for idx, (name, param) in tqdm.tqdm(enumerate(model.named_parameters())):
        # assert param.dtype is torch.float16
        if param.ndim == 2 and param.shape[0] != 1 and param.shape[1] != 1:
            r, c = param.shape
            tensor_data = list(
                map(lambda x: int(
                        "{}".format(
                                bin(
                                    int.from_bytes(
                                    struct.pack('>e', x), byteorder='big'
                                    )
                                )[3: 3+nbits]      # torch.float16 [3: 8] torch.bfloat16[3: 11]
                            ), 
                            base=2
                    ),
                    param.abs().neg().reshape(-1).tolist()
                )
            )
            tensor = torch.tensor(tensor_data, dtype=torch.int8).reshape(r, c)
            exponent[name] = tensor
    return exponent

In [3]:
### Get the exponent ###
models_hub = {
    "t5": {
        "path": "/home/styaeng/project/delta-compress/pretrained_model/t5",       ### 这里要写成下载后的模型权重文件所在的路径
        "hdlr": T5ForConditionalGeneration.from_pretrained
    },
    "gpt2": {
        "path": "/home/styaeng/project/delta-compress/pretrained_model/gpt2",       ### 这里要写成下载后的模型权重文件所在的路径
        "hdlr": GPT2LMHeadModel.from_pretrained
    },
    "llama2": {
        "path": "/home/styaeng/project/delta-compress/pretrained_model/llama2",       ### 这里要写成下载后的模型权重文件所在的路径
        "hdlr": LlamaForCausalLM.from_pretrained
    },
}

In [4]:
t5_model = models_hub['t5']['hdlr'](models_hub['t5']['path'])
t5_exponent = get_exponent(t5_model, nbits=5)

131it [00:24,  5.39it/s]


In [5]:
# gpt2_model = models_hub['gpt2']['hdlr'](models_hub['gpt2']['path'])
# gpt2_exponent = get_exponent(gpt2_model, nbits=5)

In [6]:
# llama2_model = models_hub['llama2']['hdlr'](models_hub['llama2']['path'])
# llama2_exponent = get_exponent(llama2_model, nbits=5)

In [7]:
PageSize = 4 * 1024 * 8 # bits
FP16 = 16   # bits

'''
2 bits
    [0 | 1 | 2 | 3]
3 bits
    [0 | 1 | 2 | 3 | 4 | 5 | 6 | 7]
4 bits
    [0-15]
5 bits
    [0]
'''
'''
in bits and in pages
compression ratio
saved storage space
fraction
how much pages are needed to read in average and maximum
'''
pattern = [
    [0, 1, 2, 3],
    [0, 1, 2, 3, 4, 5, 6, 7],
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    [i for i in range(2**5)],
]

In [8]:
tensor_gran_pattern = defaultdict(dict)
for k, tensor in t5_exponent.items():
    row, col = tensor.shape
    tensor_gran_pattern[k] = defaultdict(dict)
    if tensor.numel() * FP16 <= PageSize:
        tensor_gran_pattern[k] == "LTP"
    else:
        for shift in range(3, -1, -1):
            idx = 3 - shift
            pat = pattern[idx]
            tmp = tensor >> shift
            for elem in pat:
                tensor_gran_pattern[k][idx+2][elem] = torch.count_nonzero(tmp == elem).item()

In [ ]:
### parser the list tensor_gran_pattern
bits = [2, 3, 4, 5]
road_map = defaultdict(dict)
# results = defaultdict(dict)
for name, candidate in tensor_gran_pattern.items():
    print(name+':')
    max_ratio = 0
    if len(candidate) == 0:
        # compressed_model_size += 1
        road_map[name]["max_ratio"] = [
            ("page_size", 1)
        ]
        print(f"Orig_page:{1}\tunnecessary to compress")
        continue
    else:
        for bit in bits:
            stats = sorted(candidate[bit].items(), key=lambda x: x[1], reverse=True)
            total_page = {}
            cnt_stat = 0
            for pat, cnt in stats:
                after_compressed_bit = cnt * (FP16 - bit) + bit
                pattern_page = (after_compressed_bit + PageSize - 1) // PageSize
                cnt_stat += cnt
                if pattern_page == 1:
                    frac_bits = PageSize - (cnt * (FP16 - bit) + bit)
                    if frac_bits > after_compressed_bit:
                        break
                elif pattern_page > 0:
                    total_page[pat] = pattern_page
            road_map[name][bit] = total_page

            stats = dict(stats)
            original_tensor_page = (sum(list(stats.values())) * FP16 + PageSize - 1) // PageSize
            compressed_part = sum(list(total_page.values()))
            uncompressed_part = ((sum(stats.values()) - cnt_stat) * FP16 + PageSize - 1) // PageSize
            compressed_page = compressed_part + uncompressed_part
            ratio = (original_tensor_page - compressed_page)/original_tensor_page
            if ratio > max_ratio:
                max_ratio = ratio
                road_map[name]['max_ratio'] = [
                    ("bits", bit),
                    ("compression_ratio", ratio)
                ]
                print(f"Bit:{bit}\tOrig_page:{original_tensor_page}\tcompressed_page:{compressed_page}\tcompression_ratio:{ratio* 100 :.2f}")


In [36]:
tensor_gran_pattern['shared.weight'][5]

{0: 36,
 1: 32,
 2: 69,
 3: 141,
 4: 277,
 5: 614,
 6: 1187,
 7: 2317,
 8: 4729,
 9: 9277,
 10: 18669,
 11: 37359,
 12: 74808,
 13: 148944,
 14: 299011,
 15: 599325,
 16: 1197612,
 17: 2375694,
 18: 4139532,
 19: 4871473,
 20: 2505601,
 21: 162449,
 22: 373,
 23: 6,
 24: 1,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0}

In [39]:
t5_exponent['shared.weight'].shape

torch.Size([32128, 512])

In [46]:
avg_count = 0
for idx, count in enumerate(list(road_map['shared.weight'][5].values())):
    avg_count += (idx+1) * count/sum(road_map['shared.weight'][5].values())
avg_count += (idx+1) * 1/(sum(road_map['shared.weight'][5].values()))

print(avg_count * 512)

1438.9524326279616


In [37]:
road_map['shared.weight'][5]

{19: 1636,
 18: 1390,
 20: 842,
 17: 798,
 16: 403,
 15: 202,
 14: 101,
 21: 55,
 13: 50,
 12: 26,
 11: 13,
 10: 7,
 9: 4,
 8: 2}

In [29]:
compressed_ratio = 0
for k, v in t5_exponent.items():
    if len(road_map[k]['max_ratio']) == 1:
        compressed_ratio += 0
    else:
        compressed_ratio += road_map[k]['max_ratio'][-1][-1]
print(f"{compressed_ratio / len(road_map) * 100:.2f} %")

28.39 %


In [6]:
### Get the percentage in tile granularity ###
tile_gran_pattern = []
PageSize = 4 * 1024 * 8 # bits
FP16 = 16   # bits
for k, tensor in t5_exponent.items():
    tile_pattern = defaultdict(dict)
    row, col = tensor.shape
    if col * FP16 <= PageSize:
        tile_pattern[k] = "LTP"     # a.k.a of "Less Than PageSize"
    else:
        for row_idx in range(row):
            tile = tensor[row_idx]
            for shift in range(3, -1, -1):
                tile_pattern[k][row_idx] = defaultdict(dict)
                idx = 3 - shift
                pat = pattern[idx]
                tmp = tile >> shift
                for elem in pat:
                    tile_pattern[k][row_idx][shift][elem] = torch.count_nonzero(tmp == elem).item()
    tile_gran_pattern.append(tile_pattern)

In [19]:
### Get the percentage in matrix granularity ###
PageSize = 4 * 1024 * 8 # bits
FP16 = 16   # bits
mat_gran_pattern = []
for name, tensor in t5_exponent.items():
    row, col = tensor.shape
    higher_cur = 0
    lower_cur = higher_cur + PageSize // (col * FP16)
    tensor_pattern = defaultdict(dict)
    tensor_pattern['name'] = name
    while lower_cur <= row:
        tile_pattern = {}
        tile = tensor[higher_cur: lower_cur]
        for shift in range(3, -1, -1):
            idx = 3 - shift
            pat = pattern[idx]
            tmp = tile >> shift
            tile_pattern[idx+2] = {}
            for elem in pat:
                tile_pattern[idx+2][elem] = torch.count_nonzero(tmp == elem).item()
        for bit in range(2, 5):
            values = torch.tensor(list(tile_pattern[bit].values())).sort()[0]
            values = torch.sort(values, descending=True)[0]
            wasted_bits = (tile.numel() * FP16 + PageSize - 1) // PageSize * PageSize - values.sum() * FP16
            flag = False
            compressed_count = 0
            for grp_count, v in enumerate(values):
                v = v.item()
                wasted_bits -= (v * (FP16 - bit) + bit - v * FP16)
                compressed_count += v
                original_bit_count = tile.numel() * FP16
                original_page_count = (original_bit_count + PageSize - 1) // PageSize
                compressed_bit_count = (v * (FP16 - bit) + bit) * grp_count + (values.sum().item() - compressed_count) * FP16
                compressed_page_count = (compressed_bit_count + PageSize - 1) // PageSize
                frac = compressed_page_count * PageSize - compressed_bit_count
                if wasted_bits >= PageSize:
                    flag = True
                    break
            if flag:
                break
        if lower_cur == row:
            tensor_pattern[f"{higher_cur}:{lower_cur}"]["original_bit_count"] = original_bit_count
            tensor_pattern[f"{higher_cur}:{lower_cur}"]["compressed_bit_count"] = compressed_bit_count
            tensor_pattern[f"{higher_cur}:{lower_cur}"]["compressed_page_count"] = compressed_page_count
            break
        if not flag:
            if (row - higher_cur) * col * FP16 < PageSize:
                lower_cur = row
            else:
                lower_cur += 1
        else:
            if frac > compressed_bit_count:
                if (row - higher_cur) * col * FP16 < PageSize:
                    lower_cur = row
                else:
                    lower_cur += 1
            else:
                tensor_pattern[f"{higher_cur}:{lower_cur}"]["original_bit_count"] = original_bit_count
                tensor_pattern[f"{higher_cur}:{lower_cur}"]["compressed_bit_count"] = compressed_bit_count
                tensor_pattern[f"{higher_cur}:{lower_cur}"]["compressed_page_count"] = compressed_page_count
                if (row - higher_cur) * col * FP16 < PageSize:
                    lower_cur = row
                else:
                    higher_cur = lower_cur
                    lower_cur = higher_cur + PageSize // (col * FP16)
                    if lower_cur > row:
                        lower_cur = row
    mat_gran_pattern.append(tensor_pattern)

In [23]:
compressed_ratio = 0
cp = 0
ucp = 0
for idx, (k, v) in enumerate(t5_exponent.items()):
    stats = list(mat_gran_pattern[idx].values())[1:]
    for stat in stats:
        cp += stat['compressed_page_count']
    ucp += (v.numel() * FP16 // PageSize)
        # compressed_ratio += cp_r
print(f"Compression ratio = {(ucp - cp) / ucp * 100:.2f}%")


Compression ratio = 47.02%


In [23]:
### Get the percentage in matrix granularity ###
PageSize = 4 * 1024 * 8 # bits
FP16 = 16   # bits
mat_gran_pattern = []
for name, tensor in gpt2_exponent.items():
    row, col = tensor.shape
    higher_cur = 0
    lower_cur = higher_cur + PageSize // (col * FP16)
    tensor_pattern = defaultdict(dict)
    tensor_pattern['name'] = name
    while lower_cur <= row:
        tile_pattern = {}
        tile = tensor[higher_cur: lower_cur]
        for shift in range(3, -1, -1):
            idx = 3 - shift
            pat = pattern[idx]
            tmp = tile >> shift
            tile_pattern[idx+2] = {}
            for elem in pat:
                tile_pattern[idx+2][elem] = torch.count_nonzero(tmp == elem).item()
        for bit in range(2, 5):
            values = torch.tensor(list(tile_pattern[bit].values())).sort()[0]
            values = torch.sort(values, descending=True)[0]
            wasted_bits = (tile.numel() * FP16 + PageSize - 1) // PageSize * PageSize - values.sum() * FP16
            flag = False
            compressed_count = 0
            for grp_count, v in enumerate(values):
                v = v.item()
                wasted_bits -= (v * (FP16 - bit) + bit - v * FP16)
                compressed_count += v
                original_bit_count = tile.numel() * FP16
                original_page_count = (original_bit_count + PageSize - 1) // PageSize
                compressed_bit_count = (v * (FP16 - bit) + bit) * grp_count + (values.sum().item() - compressed_count) * FP16
                compressed_page_count = (compressed_bit_count + PageSize - 1) // PageSize
                frac = compressed_page_count * PageSize - compressed_bit_count
                if wasted_bits >= PageSize:
                    flag = True
                    break
        if lower_cur == row:
            tensor_pattern[f"{higher_cur}:{lower_cur}"]["original_bit_count"] = original_bit_count
            tensor_pattern[f"{higher_cur}:{lower_cur}"]["compressed_bit_count"] = compressed_bit_count
            tensor_pattern[f"{higher_cur}:{lower_cur}"]["compressed_page_count"] = compressed_page_count
            break
        if not flag:
            if (row - higher_cur) * col * FP16 < PageSize:
                lower_cur = row
            else:
                lower_cur += 1
        else:
            if frac > compressed_bit_count:
                if (row - higher_cur) * col * FP16 < PageSize:
                    lower_cur = row
                else:
                    lower_cur += 1
            else:
                tensor_pattern[f"{higher_cur}:{lower_cur}"]["original_bit_count"] = original_bit_count
                tensor_pattern[f"{higher_cur}:{lower_cur}"]["compressed_bit_count"] = compressed_bit_count
                tensor_pattern[f"{higher_cur}:{lower_cur}"]["compressed_page_count"] = compressed_page_count
                if (row - higher_cur) * col * FP16 < PageSize:
                    lower_cur = row
                else:
                    higher_cur = lower_cur
                    lower_cur = higher_cur + PageSize // (col * FP16)
                    if lower_cur > row:
                        lower_cur = row
    mat_gran_pattern.append(tensor_pattern)

In [24]:
compressed_ratio = 0
cp = 0
ucp = 0
for idx, (k, v) in enumerate(gpt2_exponent.items()):
    stats = list(mat_gran_pattern[idx].values())[1:]
    for stat in stats:
        cp += stat['compressed_page_count']
    ucp += (v.numel() * FP16 // PageSize)
        # compressed_ratio += cp_r
print(f"Compression ratio = {(ucp - cp) / ucp * 100:.2f}%")

Compression ratio = 19.54%


In [ ]:
### Get the percentage in matrix granularity ###
PageSize = 4 * 1024 * 8 # bits
FP16 = 16   # bits
mat_gran_pattern = []
for name, tensor in llama2_exponent.items():
    row, col = tensor.shape
    higher_cur = 0
    lower_cur = higher_cur + PageSize // (col * FP16)
    tensor_pattern = defaultdict(dict)
    tensor_pattern['name'] = name
    while lower_cur <= row:
        tile_pattern = {}
        tile = tensor[higher_cur: lower_cur]
        for shift in range(3, -1, -1):
            idx = 3 - shift
            pat = pattern[idx]
            tmp = tile >> shift
            tile_pattern[idx+2] = {}
            for elem in pat:
                tile_pattern[idx+2][elem] = torch.count_nonzero(tmp == elem).item()
        for bit in range(2, 5):
            values = torch.tensor(list(tile_pattern[bit].values())).sort()[0]
            values = torch.sort(values, descending=True)[0]
            wasted_bits = (tile.numel() * FP16 + PageSize - 1) // PageSize * PageSize - values.sum() * FP16
            flag = False
            compressed_count = 0
            for grp_count, v in enumerate(values):
                v = v.item()
                wasted_bits -= (v * (FP16 - bit) + bit - v * FP16)
                compressed_count += v
                original_bit_count = tile.numel() * FP16
                original_page_count = (original_bit_count + PageSize - 1) // PageSize
                compressed_bit_count = (v * (FP16 - bit) + bit) * grp_count + (values.sum().item() - compressed_count) * FP16
                compressed_page_count = (compressed_bit_count + PageSize - 1) // PageSize
                frac = compressed_page_count * PageSize - compressed_bit_count
                if wasted_bits >= PageSize:
                    flag = True
                    break
        if lower_cur == row:
            tensor_pattern[f"{higher_cur}:{lower_cur}"]["original_bit_count"] = original_bit_count
            tensor_pattern[f"{higher_cur}:{lower_cur}"]["compressed_bit_count"] = compressed_bit_count
            tensor_pattern[f"{higher_cur}:{lower_cur}"]["compressed_page_count"] = compressed_page_count
            break
        if not flag:
            if (row - higher_cur) * col * FP16 < PageSize:
                lower_cur = row
            else:
                lower_cur += 1
        else:
            if frac > compressed_bit_count:
                if (row - higher_cur) * col * FP16 < PageSize:
                    lower_cur = row
                else:
                    lower_cur += 1
            else:
                tensor_pattern[f"{higher_cur}:{lower_cur}"]["original_bit_count"] = original_bit_count
                tensor_pattern[f"{higher_cur}:{lower_cur}"]["compressed_bit_count"] = compressed_bit_count
                tensor_pattern[f"{higher_cur}:{lower_cur}"]["compressed_page_count"] = compressed_page_count
                if (row - higher_cur) * col * FP16 < PageSize:
                    lower_cur = row
                else:
                    higher_cur = lower_cur
                    lower_cur = higher_cur + PageSize // (col * FP16)
                    if lower_cur > row:
                        lower_cur = row
    mat_gran_pattern.append(tensor_pattern)

In [ ]:
compressed_ratio = 0
cp = 0
ucp = 0
for idx, (k, v) in enumerate(llama2_exponent.items()):
    stats = list(mat_gran_pattern[idx].values())[1:]
    for stat in stats:
        cp += stat['compressed_page_count']
    ucp += (v.numel() * FP16 // PageSize)
        # compressed_ratio += cp_r
print(f"Compression ratio = {(ucp - cp) / ucp * 100:.2f}%")